# Credit Card Default

In [ ]:
from time import time
import numpy as np
import matplotlib.pyplot as plt
import pandas

import sklearn
import pylab as pl
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from scipy.spatial.distance import cdist
from sklearn.preprocessing import MinMaxScaler
from sklearn.random_projection import GaussianRandomProjection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA, FastICA as ICA
from collections import defaultdict
from itertools import product

In [ ]:
np.random.seed(42)

Load Data and Pre Process Train and Test Data Sets to Create Dummy Variables

In [ ]:
#train = pandas.read_csv('C://Users//Divya//Desktop//Spring_2019//ML//Project 3//Data//csv//Pre_Process1//CreditCardDefaultTrain.csv')                      

#categorical_features_train = ['SEX', 'EDUCATION', 'MARRIAGE']
#continuous_features_train = ['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1','BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']


#for col in categorical_features_train:
    #dummies = pandas.get_dummies(train[col], prefix=col)
    #train = pandas.concat([train, dummies], axis=1)
    #train.drop(col, axis=1, inplace=True)
    
#train.head()
#train.to_csv('CreditCardDefaultTrainStd.csv') #Download Data


In [ ]:
#test = pandas.read_csv('C://Users//Divya//Desktop//Spring_2019//ML//Project 3//Data//csv//Pre_Process1//CreditCardDefaultTest.csv')

#categorical_features_test = ['SEX', 'EDUCATION', 'MARRIAGE']
#continuous_features_test = ['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1','BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']


#for col in categorical_features_test:
    #dummies = pandas.get_dummies(test[col], prefix=col)
    #test = pandas.concat([test, dummies], axis=1)
    #test.drop(col, axis=1, inplace=True)
    
#test.head()
#test.to_csv('CreditCardDefaultTestStd.csv') #Download Data

In [ ]:
train = pandas.read_csv('C://Users//Divya//Desktop//Spring_2019//ML//Project 3//Data//csv//Pre_Process2//CreditCardDefaultTrainStd.csv')                      
test = pandas.read_csv('C://Users//Divya//Desktop//Spring_2019//ML//Project 3//Data//csv//Pre_Process2//CreditCardDefaultTestStd.csv')

#Credit Card Train Data Features and Labels
train_data  = train.iloc[:,0:29]
train_data = train_data.values
mms = MinMaxScaler()
mms.fit(train_data)
data = mms.transform(train_data)
labels = train['defaultPayment_label']


#Credit CardTest Data Features and Labels
test_data  = test.iloc[:,0:29]
test_data = test_data.values
mms2 = MinMaxScaler()
mms2.fit(test_data)
data_test = mms.transform(test_data)
labels_test = test['defaultPayment_label']

sample_size = 50


In [ ]:
#Elbow for Kmeans
Sum_of_squared_distances = []
K = range(1,50)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(data)
    Sum_of_squared_distances.append(km.inertia_)
    
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum of Squared Distances')
plt.title('K_Means: Credit Card Data Set Elbow Method For Optimal k')
plt.show()

In [ ]:
#Elbow for Expectation Max
silhouette_score = []

K = range(2,20)
for k in K:
    gm = GaussianMixture(n_components= k, n_init=2, random_state=0).fit(data)
    gmPredicted = gm.predict(data)
    silhouette_score.append(metrics.silhouette_score(data, gmPredicted, metric='euclidean', sample_size=sample_size))
    
plt.plot(K, silhouette_score, 'bx-')
plt.xlabel('k')
plt.ylabel('Silhouette Score')
plt.title('Expectation Maximization: Credit Card Data Set Elbow Method For Optimal k')
plt.show()

In [ ]:
def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print('% 9s     %.2fs       %.3f        %.3f        %.3f        %.3f        %.3f         %.3f       %.3f'
          % (name, (time() - t0), 
             metrics.homogeneity_score(labels, estimator.predict(data)),
             metrics.completeness_score(labels, estimator.predict(data)),
             metrics.v_measure_score(labels, estimator.predict(data)),
             metrics.adjusted_rand_score(labels, estimator.predict(data)),
             metrics.adjusted_mutual_info_score(labels,  estimator.predict(data)),
             metrics.silhouette_score(data, estimator.predict(data),metric='euclidean',sample_size=sample_size),
             float(sum(estimator.predict(data) == labels))/float(len(labels))))


In [ ]:
n_digits_i = [10,20,30,40,50]
for i in range(5):
    bench_k_means(KMeans(init='k-means++', n_clusters=n_digits_i[i], n_init=10),name="k-means++", data=data)
for i in range(5):
    bench_k_means(GaussianMixture(n_components=n_digits_i[i],random_state=0),name="GaussianMixture", data=data)

In [ ]:
PCA_data = PCA().fit(data)

#Plotting the Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(np.cumsum(PCA_data.explained_variance_ratio_*100))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('PCA: Credit Card Dataset Explained Variance')
plt.show()


In [ ]:
PCA_data = PCA(n_components = 5, whiten=False)
PCA_data.fit(data)
PCA_data_trans = PCA_data.transform(data)
PCA_data_trans_test = PCA_data.transform(data_test)

#Plots
per_var = np.round(PCA_data.explained_variance_ratio_* 100, decimals=1)
print("Original Credit Card Data Set Number of Rows and Columns:", data.shape)
print("Original Credit Card Data Set Number of Rows and Columns:", data_test.shape)
print("PCA Credit Card Train Data Set Number of Rows and Columns:", PCA_data_trans.shape)
print("PCA Credit Card Test Data Set Number of Rows and Columns:", PCA_data_trans_test.shape)

#Plotting the Cumulative Summation of the Explained Variance
plt.figure(figsize=(8,6))
plt.scatter(PCA_data_trans[:,0],PCA_data_trans[:,1], c = train['defaultPayment_label'])
plt.title('PCA: Credit Card Dataset Graph')
plt.xlabel('PC1 - {0}%'.format(per_var[0]))
plt.ylabel('PC2 - {0}%'.format(per_var[1]))

In [ ]:
def run_ICA(X,y,title):
    
    dims = list(np.arange(2,(X.shape[1]-1),3))
    dims.append(X.shape[1])
    ica = ICA(random_state=5)
    kurt = []

    for dim in dims:
        ica.set_params(n_components=dim)
        tmp = ica.fit_transform(X)
        tmp = pandas.DataFrame(tmp)
        tmp = tmp.kurt(axis=0)
        kurt.append(tmp.abs().mean())

    plt.figure()
    plt.title("ICA Kurtosis: "+ title)
    plt.xlabel("Independent Components")
    plt.ylabel("Avg Kurtosis Across IC")
    plt.plot(dims, kurt, 'b-')
    plt.grid(False)
    plt.show()

run_ICA(data,labels,"ICA: Credit Card Dataset Avergae Kurtosis")

In [ ]:
ICA_data = FastICA(n_components = 25)
ICA_data.fit(data)
ICA_data_trans = ICA_data.transform(data)
ICA_data_trans_test = ICA_data.transform(data_test)


In [ ]:
RP_data = GaussianRandomProjection(n_components=5, eps=0.1)
RP_data_trans = RP_data.fit_transform(data)
RP_data_trans_test = RP_data.fit_transform(data_test)

plt.figure(figsize=(8,6))
plt.scatter(RP_data_trans[:,0],RP_data_trans[:,1], c = train['defaultPayment_label'])
plt.title('RP: Credit Card Dataset Graph')
plt.xlabel('RP1')
plt.ylabel('RP2')

In [ ]:
# Create and run an LDA
LDA_data = LinearDiscriminantAnalysis(n_components=1)
LDA_data_trans = LDA_data.fit_transform(data, labels)
LDA_data_trans_test = LDA_data.fit_transform(data_test, labels_test)


plt.figure(figsize=(8,6))
plt.scatter(LDA_data_trans[:,0], labels, c = train['defaultPayment_label'])
plt.title('LDA: Credit Card Dataset Graph')
plt.xlabel('LDA1')



In [ ]:
#Elbow for Kmeans
Sum_of_squared_distances = []
K = range(1,30)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(LDA_data_trans)
    Sum_of_squared_distances.append(km.inertia_)
    
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum of Squared Distances')
plt.title('K_Means/LDA: Credit Card Data Set Elbow Method For Optimal k')
plt.show()

In [ ]:
#Elbow for Expectation Max
silhouette_score = []

K = range(2,10)
for k in K:
    gm = GaussianMixture(n_components= k, n_init=2, random_state=0).fit(LDA_data_trans)
    gmPredicted = gm.predict(LDA_data_trans)
    silhouette_score.append(metrics.silhouette_score(LDA_data_trans, gmPredicted, metric='euclidean', sample_size=sample_size))
    
plt.plot(K, silhouette_score, 'bx-')
plt.xlabel('k')
plt.ylabel('Silhouette Score')
plt.title('Expectation Maximization/LDA: Adult Data Set Elbow Method For Optimal k')
plt.show()

In [ ]:
#Run datasets
n_digits_i = [5]
for i in range(1):
    bench_k_means(KMeans(init='k-means++', n_clusters=n_digits_i[i], n_init=10),name="k-means++", data=LDA_data_trans)


In [ ]:
n_digits_i = [2]
for i in range(1):
    bench_k_means(GaussianMixture(n_components=n_digits_i[i],random_state=0),name="GaussianMixture", data=ICA_data_trans)

In [ ]:
#import time
#without transformation
#start_time = time.time()
data_nn = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(4,), random_state=1, momentum = 0.1, learning_rate_init = 0.1)

#4,4
#4,4,4,4,4
#4,4,4,4,4,4,4,4,4,4
#4,4,4,4,4,4,4,4,4,4,4,4,4,4,4 

##Test###
#data_nn.fit(data, labels)  
#data_train_pred = data_nn.predict(data)
#float(sum(data_train_pred == labels))/float(len(labels))
#print("--- %s training seconds ---" % (time.time() - start_time))

##Train###
#start_time2 = time.time()
#data_test_pred = data_nn.predict(data_test)
#float(sum(data_test_pred == labels_test))/float(len(labels_test))
#print("--- %s testing seconds ---" % (time.time() - start_time2))


In [ ]:
#PCA Train
data_nn.fit(PCA_data_trans, labels)  
data_train_pred_PCA = data_nn.predict(PCA_data_trans)
float(sum(data_train_pred_PCA == labels))/float(len(labels))

In [ ]:
#PCA Test
data_test_pred_PCA = data_nn.predict(PCA_data_trans_test)
float(sum(data_test_pred_PCA == labels_test))/float(len(labels_test))

In [ ]:
#ICA Train
data_nn.fit(ICA_data_trans, labels)  
data_train_pred_ICA = data_nn.predict(ICA_data_trans)
float(sum(data_train_pred_ICA == labels))/float(len(labels))

In [ ]:
#ICA Test
data_test_pred_ICA = data_nn.predict(ICA_data_trans_test)
float(sum(data_test_pred_ICA == labels_test))/float(len(labels_test))

In [ ]:
#RP Train
data_nn.fit(RP_data_trans, labels)  
data_train_pred_RP = data_nn.predict(RP_data_trans)
float(sum(data_train_pred_RP == labels))/float(len(labels))

In [ ]:
#RP Test
data_test_pred_RP = data_nn.predict(RP_data_trans_test)
float(sum(data_test_pred_RP == labels_test))/float(len(labels_test))

In [ ]:
#LDA Train
data_nn.fit(LDA_data_trans, labels)  
data_train_pred_LDA = data_nn.predict(LDA_data_trans)
float(sum(data_train_pred_LDA == labels))/float(len(labels))

In [ ]:
#LDA Test
data_test_pred_LDA = data_nn.predict(LDA_data_trans_test)
float(sum(data_test_pred_LDA == labels_test))/float(len(labels_test))

In [ ]:
kmeans_cluster = KMeans(n_clusters=10)
pca_train_dataframe["cluster"] = kmeans_cluster.fit_predict(pca_train_dataframe[pca_train_dataframe.columns[0:]])
pca_test_dataframe["cluster"] = kmeans_cluster.fit_predict(pca_test_dataframe[pca_test_dataframe.columns[0:]])

kmeans_cluster = KMeans(n_clusters=20)
ica_train_dataframe["cluster"] = kmeans_cluster.fit_predict(ica_train_dataframe[ica_train_dataframe.columns[0:]])
ica_test_dataframe ["cluster"] = kmeans_cluster.fit_predict(ica_test_dataframe [ica_test_dataframe.columns[0:]])

kmeans_cluster = KMeans(n_clusters=10)
rp_train_dataframe["cluster"] = kmeans_cluster.fit_predict(rp_train_dataframe[rp_train_dataframe.columns[0:]])
rp_test_dataframe ["cluster"] = kmeans_cluster.fit_predict(rp_test_dataframe [rp_test_dataframe.columns[0:]])

kmeans_cluster = KMeans(n_clusters=5)
lda_train_dataframe["cluster"] = kmeans_cluster.fit_predict(lda_train_dataframe[lda_train_dataframe.columns[0:]])
lda_test_dataframe["cluster"] = kmeans_cluster.fit_predict(lda_test_dataframe [lda_test_dataframe.columns[0:]])

In [ ]:
data_nn = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(4,), random_state=1, momentum = 0.1, learning_rate_init = 0.1)

In [ ]:
#PCA Train
data_nn.fit(pca_train_dataframe, labels)  
data_train_pred_PCA = data_nn.predict(pca_train_dataframe)
float(sum(data_train_pred_PCA == labels))/float(len(labels))

In [ ]:
#PCA Test
data_test_pred_PCA = data_nn.predict(pca_test_dataframe)
float(sum(data_test_pred_PCA == labels_test))/float(len(labels_test))

In [ ]:
#ICA Train
data_nn.fit(ica_train_dataframe, labels)  
data_train_pred_ICA = data_nn.predict(ica_train_dataframe)
float(sum(data_train_pred_ICA == labels))/float(len(labels))

In [ ]:
#ICA Test
data_test_pred_ICA = data_nn.predict(ica_test_dataframe)
float(sum(data_test_pred_ICA == labels_test))/float(len(labels_test))

In [ ]:
#RP Train
data_nn.fit(rp_train_dataframe, labels)  
data_train_pred_RP = data_nn.predict(rp_train_dataframe)
float(sum(data_train_pred_RP == labels))/float(len(labels))

In [ ]:
#RP Test
data_test_pred_RP = data_nn.predict(rp_test_dataframe)
float(sum(data_test_pred_RP == labels_test))/float(len(labels_test))

In [ ]:
#LDA Train
data_nn.fit(lda_train_dataframe, labels)  
data_train_pred_LDA = data_nn.predict(lda_train_dataframe)
float(sum(data_train_pred_LDA == labels))/float(len(labels))

In [ ]:
#LDA Test
data_test_pred_LDA = data_nn.predict(lda_test_dataframe)
float(sum(data_test_pred_LDA == labels_test))/float(len(labels_test))